In [ ]:
# 4.12.7

import numpy as np
import math
import matplotlib.pyplot as plt

eps = 10**-5

$
\begin{equation*}
 \begin{cases}
   sin(x) - y = 1.32 
   \\
   cos(y) - x = -0.85
 \end{cases}
\end{equation*}
$ \
для локолизации разобьем плоскость на периоды синуса и косинуса. Пересекаются в
$ \{ -0.15 < x < 1.85; -2.32 < y < -0.32 \} $, шаг проверки - $ \frac{\pi}{2}$ 

In [ ]:
loc_x = [[-np.pi / 2, 0.0], [0.0, np.pi / 2], [np.pi / 2, np.pi]]
loc_y = [[-np.pi, -np.pi / 2], [-np.pi / 2, 0.0]]

In [ ]:
func1 = lambda x: np.sin(x) - 1.32
func2 = lambda y: np.cos(y) + 0.85
step = 0.1

print("------------------start localization---------------------")
res_locs_x = np.array([[]])
for lx in loc_x:
  for ly in loc_y:
    print("start in ", lx, ly)
    x_ = np.arange(start=lx[0], stop=lx[1], step=step)
    sign_ = 0
    for i in range(0, len(x_)):
      yi = func1(x_[i])
      # проверяем принадлежит ли текущему отрезку локализации по y
      if yi < ly[0] or yi > ly[1]:
        continue;

      cur_sign_ = np.sign(func2(yi) - x_[i])

      if sign_ == 0:
        sign_ = cur_sign_

      if cur_sign_ != sign_ and cur_sign_ != 0:
        print("find!")
        res_locs_x = np.append(res_locs_x, [x_[i - 1], x_[i]])
        sign_ = cur_sign_
print("-------------------end localization----------------------")
print("xbox: ", res_locs_x)
print("ybox: ", list(map(func1, res_locs_x)))

------------------start localization---------------------
start in  [-1.5707963267948966, 0.0] [-3.141592653589793, -1.5707963267948966]
start in  [-1.5707963267948966, 0.0] [-1.5707963267948966, 0.0]
start in  [0.0, 1.5707963267948966] [-3.141592653589793, -1.5707963267948966]
start in  [0.0, 1.5707963267948966] [-1.5707963267948966, 0.0]
start in  [1.5707963267948966, 3.141592653589793] [-3.141592653589793, -1.5707963267948966]
start in  [1.5707963267948966, 3.141592653589793] [-1.5707963267948966, 0.0]
find!
-------------------end localization----------------------
xbox:  [1.77079633 1.87079633]
ybox:  [-0.33993342215875844, -0.3646635108743941]


# МПИ

В областях локализации производные строго меньше единицы. 

In [ ]:
# Функции для проверки (равны нулю в решениях)
func1_full = lambda x, y : np.sin(x) - y - 1.32
func2_full = lambda x, y : np.cos(y) - x + 0.85

# Проходим по всем облостям локализации
for i in range(0, len(res_locs_x), 2):
  print("------------------start MPI---------------------")
  lx = [ res_locs_x[i], res_locs_x[i + 1] ]
  ly = [ func1(lx[0]), func1(lx[1]) ]

  iter_counter = 0
  sol = [(lx[0] + lx[1]) / 2.0, (ly[0] + ly[1]) / 2.0 ]
  while(np.linalg.norm(np.array([ func1_full(sol[0], sol[1]), func2_full(sol[0], sol[1]) ])) > eps):
    sol = np.array([ func2(sol[1]), func1(sol[0]) ])
    iter_counter += 1
  print("solution ")
  print(sol)
  print("\nnum iterations ", iter_counter)
  print("-------------------end MPI----------------------")

------------------start MPI---------------------
solution 
[ 1.79133971 -0.34421826]

num iterations  7
-------------------end MPI----------------------


# Метод Ньютона


In [ ]:
# Проходим по всем областям локализации
for i in range(0, len(res_locs_x), 2):
  print("------------------start N---------------------")
  lx = [ res_locs_x[i], res_locs_x[i + 1] ]
  ly = [ func1(lx[0]), func1(lx[1]) ]

  iter_counter = 0
  sol = [(2*lx[0] + lx[1]) / 3.0, (ly[0] + ly[1]) / 2.0 ]
  J = np.matrix(np.zeros([2, 2]))
  J[0, 0] = np.cos(sol[0])
  J[1, 1] = - np.sin(sol[1])
  J[0, 1] = -1
  J[1, 0] = -1
  J_inverse = np.linalg.inv(J)
  while(np.linalg.norm(np.array([ func1_full(sol[0], sol[1]) , func2_full(sol[0], sol[1]) ])) > eps):
    F = np.matrix([func1_full(sol[0], sol[1]), func2_full(sol[0], sol[1])])

    sol = np.array((np.matrix(sol).T - J_inverse * F.T).T)[0]
    print(sol)
    iter_counter += 1
  print("solution ")
  print(sol)
  print("\nnum iterations ", iter_counter)
  print("-------------------end N----------------------")

------------------start N---------------------
[ 1.79139245 -0.34415382]
[ 1.79133835 -0.34422031]
solution 
[ 1.79133835 -0.34422031]

num iterations  2
-------------------end N----------------------
